In [1]:
import os
import time
import copy
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

from model import EnvironNet
from utils import num_params
from train_test import test_model
from data import AudioDataset, collate_fn

In [2]:
df = pd.read_csv("ESC-50/meta/esc50.csv")
df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
batch_size = 32

if device == "cuda":
    num_workers = 0
    pin_memory = True

elif device == "cpu":
    num_workers = 0
    pin_memory = False

weights_path = "weights/baseline.pth.tar"
weights = torch.load(weights_path)
criterion = nn.CrossEntropyLoss()
accuracies = {}

for i in range(1, 6):
    model = EnvironNet(
        in_channels = 1,
        n_classes = 50,
        kernel_size = (3, 3),
        num_subspecs = 2,
        affine = True,
        dropout = 0.
    ).to(device)
    model = nn.DataParallel(model)
    model.load_state_dict(weights[f"fold_{i}"])

    optimizer = optim.SGD(
        model.parameters(),
        lr = 0.0005,
        momentum = 0.9,
        weight_decay = 0.001,
        nesterov = True
    )

    df_test = df[df["fold"] == i]
    df_test.reset_index(drop=True, inplace=True)

    test_set = AudioDataset(
        df_test,
        "ESC-50/audio",
    )

    test_loader = DataLoader(
        test_set,
        batch_size = batch_size,
        shuffle = True,
        collate_fn = collate_fn,
        num_workers = num_workers,
        pin_memory = pin_memory,
    )
  
    model, cache = test_model(model, test_loader, optimizer, criterion, device)
    _, test_accuracy, *_ = cache.values()
    accuracies[f"fold_{i+1}"] = test_accuracy

Device: cuda
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s
Testing complete in 0m 3s


In [4]:
# Accuracies sometimes vary per run
accs = np.array(list(accuracies.values()))
for i, acc in enumerate(accs):
    print(f"Fold {i+1} accuracy: {acc*100:.1f}%")
print(f"Five fold cross-validation accuracy: {accs.mean()*100:.1f}% +-",
    f"{accs.std()*100:.1f}%")

Fold 1 accuracy: 77.8%
Fold 2 accuracy: 77.0%
Fold 3 accuracy: 78.5%
Fold 4 accuracy: 80.2%
Fold 5 accuracy: 72.8%
Five fold cross-validation accuracy: 77.2% +- 2.5%
